In [1]:
import pandas as pd
from open_geodata import geo

from sp_mpsp_divadmin.paths import input_path_tab, output_path_tabs

In [2]:
df_mun = geo.load_dataset('tab.sp.tab_municipio_nome')

df_mun.info()
df_mun.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 645 entries, 0 to 644
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id_municipio    645 non-null    int64 
 1   municipio_nome  645 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.2+ KB


,id_municipio,municipio_nome
0,3500105,Adamantina
1,3500204,Adolfo
2,3500303,Aguaí
3,3500402,Águas da Prata
4,3500501,Águas de Lindóia


In [3]:
# Dataframe
df_ids = pd.read_csv(
    input_path_tab / 'tab_ids_municipio.csv',
    # usecols=['id_municipio', 'id_gaema'],
    usecols=['id_municipio', 'id_gaema', 'id_rp'],
)

# Results
df_ids.info()
df_ids.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 645 entries, 0 to 644
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_municipio  645 non-null    int64  
 1   id_gaema      265 non-null    float64
 2   id_rp         380 non-null    float64
dtypes: float64(2), int64(1)
memory usage: 15.2 KB


,id_municipio,id_gaema,id_rp
0,3500105,NaN,11.0
1,3500204,NaN,10.0
2,3500303,NaN,14.0
3,3500402,NaN,14.0
4,3500501,NaN,14.0


In [4]:
# Merge Municípios
df = pd.merge(
    df_mun,
    df_ids,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio',
)

# Results
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 645 entries, 0 to 644
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_municipio    645 non-null    int64  
 1   municipio_nome  645 non-null    object 
 2   id_gaema        265 non-null    float64
 3   id_rp           380 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 20.3+ KB


,id_municipio,municipio_nome,id_gaema,id_rp
0,3500105,Adamantina,NaN,11.0
1,3500204,Adolfo,NaN,10.0
2,3500303,Aguaí,NaN,14.0
3,3500402,Águas da Prata,NaN,14.0
4,3500501,Águas de Lindóia,NaN,14.0


<br>

## GAEMA


In [5]:
mask = (~df['id_gaema'].isnull()) & (df['id_rp'].isnull())
df[mask]

,id_municipio,municipio_nome,id_gaema,id_rp
6,3500600,Águas de São Pedro,11.0,NaN
7,3500709,Agudos,7.0,NaN
9,3500808,Alfredo Marcondes,6.0,NaN
11,3501004,Altinópolis,5.0,NaN
13,3501152,Alumínio,12.0,NaN
...,...,...,...,...
633,3556354,Vargem,10.0,NaN
634,3556404,Vargem Grande do Sul,5.0,NaN
636,3556503,Várzea Paulista,9.0,NaN
638,3556701,Vinhedo,9.0,NaN


In [6]:
# Dataframe GAEMAs
df_gaema = pd.read_csv(
    input_path_tab / 'tab_gaema.csv',
    usecols=['id', 'nome'],
)
df_gaema = df_gaema.add_prefix('gaema_')

# df_gaema['gaema_nome'] = df_gaema['gaema_nome'].add_prefix('')
df_gaema['gaema_nome'].replace('Núcleo', '', regex=True, inplace=True)
df_gaema['gaema_nome'] = df_gaema['gaema_nome'].apply(lambda x: f'GAEMA - {x}')
df_gaema['unidade'] = df_gaema['gaema_nome']

# Results
df_gaema.info()
df_gaema.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   gaema_id    12 non-null     int64 
 1   gaema_nome  12 non-null     object
 2   unidade     12 non-null     object
dtypes: int64(1), object(2)
memory usage: 416.0+ bytes


,gaema_id,gaema_nome,unidade
0,1,GAEMA - Vale do Paraíba,GAEMA - Vale do Paraíba
1,2,GAEMA - Vale do Ribeira,GAEMA - Vale do Ribeira
2,3,GAEMA - Baixada Santista,GAEMA - Baixada Santista
3,4,GAEMA - Litoral Norte,GAEMA - Litoral Norte
4,5,GAEMA - Pardo,GAEMA - Pardo


In [7]:
# Merge GAEMAs
df_gaema_merged = pd.merge(
    df[mask],
    df_gaema,
    how='left',
    left_on='id_gaema',
    right_on='gaema_id',
)


df_gaema_merged.info()
df_gaema_merged.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_municipio    265 non-null    int64  
 1   municipio_nome  265 non-null    object 
 2   id_gaema        265 non-null    float64
 3   id_rp           0 non-null      float64
 4   gaema_id        265 non-null    int64  
 5   gaema_nome      265 non-null    object 
 6   unidade         265 non-null    object 
dtypes: float64(2), int64(2), object(3)
memory usage: 14.6+ KB


,id_municipio,municipio_nome,id_gaema,id_rp,gaema_id,gaema_nome,unidade
0,3500600,Águas de São Pedro,11.0,NaN,11,GAEMA - PCJ-Piracicaba,GAEMA - PCJ-Piracicaba
1,3500709,Agudos,7.0,NaN,7,GAEMA - Médio Paranapanema,GAEMA - Médio Paranapanema
2,3500808,Alfredo Marcondes,6.0,NaN,6,GAEMA - Pontal do Paranapanema,GAEMA - Pontal do Paranapanema
3,3501004,Altinópolis,5.0,NaN,5,GAEMA - Pardo,GAEMA - Pardo
4,3501152,Alumínio,12.0,NaN,12,GAEMA - Tietê/Sorocaba,GAEMA - Tietê/Sorocaba


<br>

## Rede Protetiva


In [8]:
mask = (df['id_gaema'].isnull()) & (~df['id_rp'].isnull())
df[mask]

,id_municipio,municipio_nome,id_gaema,id_rp
0,3500105,Adamantina,NaN,11.0
1,3500204,Adolfo,NaN,10.0
2,3500303,Aguaí,NaN,14.0
3,3500402,Águas da Prata,NaN,14.0
4,3500501,Águas de Lindóia,NaN,14.0
...,...,...,...,...
639,3556800,Viradouro,NaN,4.0
640,3556909,Vista Alegre do Alto,NaN,7.0
641,3556958,Vitória Brasil,NaN,9.0
643,3557105,Votuporanga,NaN,7.0


In [9]:
# Dataframe Rede Protetiva
df_rp = pd.read_csv(
    input_path_tab / 'tab_redeprotetiva.csv',
    usecols=['id', 'nome'],
)
df_rp = df_rp.add_prefix('rp_')

# df_gaema['gaema_nome'] = df_gaema['gaema_nome'].add_prefix('')
# df_rp['rp_nome'].replace('Núcleo', '', regex=True, inplace=True)
df_rp['rp_nome'] = df_rp['rp_nome'].apply(lambda x: f'Rede Protetiva - {x}')
df_rp['unidade'] = df_rp['rp_nome']

# Results
df_rp.info()
df_rp.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   rp_id    16 non-null     int64 
 1   rp_nome  16 non-null     object
 2   unidade  16 non-null     object
dtypes: int64(1), object(2)
memory usage: 512.0+ bytes


,rp_id,rp_nome,unidade
0,1,Rede Protetiva - Mantiqueira,Rede Protetiva - Mantiqueira
1,2,Rede Protetiva - Sapucaí/Grande,Rede Protetiva - Sapucaí/Grande
2,3,Rede Protetiva - Tietê/Sorocaba,Rede Protetiva - Tietê/Sorocaba
3,4,Rede Protetiva - Baixo Pardo/Grande,Rede Protetiva - Baixo Pardo/Grande
4,5,Rede Protetiva - Tietê/Jacaré,Rede Protetiva - Tietê/Jacaré


In [10]:
# Merge GAEMAs
df_rp_merged = pd.merge(
    df[mask],
    df_rp,
    how='left',
    left_on='id_rp',
    right_on='rp_id',
)

df_rp_merged.info()
df_rp_merged.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_municipio    380 non-null    int64  
 1   municipio_nome  380 non-null    object 
 2   id_gaema        0 non-null      float64
 3   id_rp           380 non-null    float64
 4   rp_id           380 non-null    int64  
 5   rp_nome         380 non-null    object 
 6   unidade         380 non-null    object 
dtypes: float64(2), int64(2), object(3)
memory usage: 20.9+ KB


,id_municipio,municipio_nome,id_gaema,id_rp,rp_id,rp_nome,unidade
0,3500105,Adamantina,NaN,11.0,11,Rede Protetiva - Aguapeí/Peixe,Rede Protetiva - Aguapeí/Peixe
1,3500204,Adolfo,NaN,10.0,10,Rede Protetiva - Baixo Tietê,Rede Protetiva - Baixo Tietê
2,3500303,Aguaí,NaN,14.0,14,Rede Protetiva - Alto Mogi,Rede Protetiva - Alto Mogi
3,3500402,Águas da Prata,NaN,14.0,14,Rede Protetiva - Alto Mogi,Rede Protetiva - Alto Mogi
4,3500501,Águas de Lindóia,NaN,14.0,14,Rede Protetiva - Alto Mogi,Rede Protetiva - Alto Mogi


In [11]:
df_final = pd.concat([df_gaema_merged, df_rp_merged])
df_final.drop(
    ['id_gaema', 'id_rp', 'gaema_id', 'gaema_nome', 'rp_id', 'rp_nome'],
    axis=1,
    inplace=True,
    errors='ignore',
)

df_final.info()
df_final.head()

<class 'pandas.core.frame.DataFrame'>
Index: 645 entries, 0 to 379
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id_municipio    645 non-null    int64 
 1   municipio_nome  645 non-null    object
 2   unidade         645 non-null    object
dtypes: int64(1), object(2)
memory usage: 20.2+ KB


,id_municipio,municipio_nome,unidade
0,3500600,Águas de São Pedro,GAEMA - PCJ-Piracicaba
1,3500709,Agudos,GAEMA - Médio Paranapanema
2,3500808,Alfredo Marcondes,GAEMA - Pontal do Paranapanema
3,3501004,Altinópolis,GAEMA - Pardo
4,3501152,Alumínio,GAEMA - Tietê/Sorocaba


In [12]:
df_final['id_municipio_6d'] = df_final['id_municipio'].astype(str).str[:6]
df_final

,id_municipio,municipio_nome,unidade,id_municipio_6d
0,3500600,Águas de São Pedro,GAEMA - PCJ-Piracicaba,350060
1,3500709,Agudos,GAEMA - Médio Paranapanema,350070
2,3500808,Alfredo Marcondes,GAEMA - Pontal do Paranapanema,350080
3,3501004,Altinópolis,GAEMA - Pardo,350100
4,3501152,Alumínio,GAEMA - Tietê/Sorocaba,350115
...,...,...,...,...
375,3556800,Viradouro,Rede Protetiva - Baixo Pardo/Grande,355680
376,3556909,Vista Alegre do Alto,Rede Protetiva - Turvo/Grande,355690
377,3556958,Vitória Brasil,Rede Protetiva - São José dos Dourados,355695
378,3557105,Votuporanga,Rede Protetiva - Turvo/Grande,355710


In [13]:
df_final.to_csv(
    output_path_tabs / 'tab_gaemas_rps.csv',
    index=False,
)